In [1]:
!pip install datasets
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 15.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system ==

In [3]:
from transformers import T5Tokenizer, T5ForConditionalGeneration, TrainingArguments, Trainer
from transformers import RobertaTokenizer
from datasets import load_dataset
import torch, evaluate

dataset = load_dataset("code_search_net", "python")
train_data = dataset["train"].shuffle(seed=42).select(range(20000))
val_data = dataset["validation"].select(range(2000))



tokenizer = RobertaTokenizer.from_pretrained("Salesforce/codet5-base")
model = T5ForConditionalGeneration.from_pretrained("Salesforce/codet5-base")
model.config.decoder_start_token_id = tokenizer.pad_token_id


max_input = 256
max_target = 64

def preprocess(example):
    input_text = "Generate comment: " + example["func_code_string"]
    target_text = example["func_documentation_string"]
    model_inputs = tokenizer(input_text, max_length=max_input, padding="max_length", truncation=True)
    labels = tokenizer(target_text, max_length=max_target, padding="max_length", truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

train_dataset = train_data.map(preprocess, remove_columns=train_data.column_names)
val_dataset = val_data.map(preprocess, remove_columns=val_data.column_names)



/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/12.9k [00:00<?, ?B/s]

code_search_net.py:   0%|          | 0.00/8.44k [00:00<?, ?B/s]

The repository for code_search_net contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/code_search_net.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


python.zip:   0%|          | 0.00/941M [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/1.57k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/892M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

Map:   0%|          | 0/20000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [4]:
training_args = TrainingArguments(
    output_dir="./codet5-results",
    num_train_epochs=10,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    logging_dir="./logs"
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer
)

trainer.train()

model.save_pretrained("./codet5-comment-generator")
tokenizer.save_pretrained("./codet5-comment-generator")

def generate_comment(code: str):
    input_text = "Generate comment: " + code
    inputs = tokenizer(input_text, return_tensors="pt", truncation=True, padding="max_length", max_length=1024)
    input_ids = inputs.input_ids.to(model.device)

    output_ids = model.generate(
        input_ids,
        max_length=64,
        num_beams=4,                
        early_stopping=True,
        repetition_penalty=2.0,
        length_penalty=1.0,
        no_repeat_ngram_size=2,
        eos_token_id=tokenizer.eos_token_id
    )

    return tokenizer.decode(output_ids[0], skip_special_tokens=True)


print(generate_comment("def add(x, y): return x + y"))


<ipython-input-4-0c330b5b9a5f>:12: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: parthmundhwa004 (parthmundhwa004-fanshawe-college) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Step,Training Loss
500,0.090900
1000,0.028000
1500,0.024300
2000,0.021600
2500,0.023100
3000,0.020300
3500,0.017200
4000,0.013200
4500,0.015200
5000,0.013600


Step,Training Loss
500,0.090900
1000,0.028000
1500,0.024300
2000,0.021600
2500,0.023100
3000,0.020300
3500,0.017200
4000,0.013200
4500,0.015200
5000,0.013600


x + y


In [5]:
print(generate_comment("def multiply(x, y): return x * y * y * y"))
print(generate_comment("def square_root(x): return x ** 0.5"))
print(generate_comment("def is_even(n): return n % 2 == 0"))
print(generate_comment("def factorial(n): return 1 if n == 0 else n * factorial(n - 1)"))

Square multiplication between points and times, using multiplying algorithm.
x ** 0.5
is_even(n) = True if n % 2 == 0 else return false
1 if n == 0 else n * factorial(n - 1)
